In [43]:
from typing import Dict, Tuple, Union, List, Any

In [2]:
import random

def fuzzer(
    max_length: int = 100,
    char_start: int = 32,
    char_range: int = 32
) -> str:
    string_length = random.randrange(0, max_length + 1)
    out = ""
    for i in range(0, string_length):
        out += chr(random.randrange(char_start, char_start + char_range))
    return out

fuzzer()

"!=*!&7:2%=?7/+7$+$,$-!$,-6.>!#9(?!-887(?6?59?$.38<9'(/*%<+7'+("

In [45]:
fuzzer(1000, ord('a'), 26)

'kbmaakoawinlgrfkltbrsdxxtwctdkfntziljdhfgteafamdtbzsznmmtqnesnsbejrcinxfmxmdxitzjhhpetndhkrjfqhlof'

In [46]:
import os
import tempfile

basename = "input.txt"
tempdir = tempfile.mkdtemp()
FILE = os.path.join(tempdir, basename)
print(FILE)

data = fuzzer()
with open(FILE, "w") as f:
    f.write(data)

contents = open(FILE).read()
print(contents)
assert(contents == data)

/tmp/tmptuo2y2hf/input.txt
3=777?0$-),73';(,&(%($47%"?17!03'!#'5:=0?:9;%>92%:' )/'%=75)6=+1+8:-5,1;.=7#'/<68& :/:+5%+;


In [47]:
import subprocess

program = "bc"
with open(FILE, "w") as f:
    f.write("2 + 2\n")
result = subprocess.run(
    [program, FILE],
    stdin=subprocess.DEVNULL,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    universal_newlines=True
)
result.stdout

'4\n'

In [48]:
trials = 100
program = "bc"

runs = []

for i in range(trials):
    data = fuzzer()
    with open(FILE, "w") as f:
        f.write(data)
    result = subprocess.run([program, FILE],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)
    runs.append((data, result))

KeyboardInterrupt: 

In [ ]:
def crash_if_too_long(s):
    buffer = "Thursday"
    if len(s) > len(buffer):
        raise ValueError
    
from ExpectError import ExpectError

trials = 100
with ExpectError():
    for i in range(trials):
        s = fuzzer()
        crash_if_too_long(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_29479/3818397696.py", line 12, in <module>
    crash_if_too_long(s)
    ~~~~~~~~~~~~~~~~~^^^
  File "/tmp/ipykernel_29479/3818397696.py", line 4, in crash_if_too_long
    raise ValueError
ValueError (expected)


In [5]:
import sys
from ExpectError import ExpectTimeout


def hang_if_no_space(s):
    i = 0
    while True:
        if i < len(s):
            if s[i] == ' ':
                break
        i += 1


trials = 100
with ExpectTimeout(2):
    for i in range(trials):
        s = fuzzer()
        hang_if_no_space(s)

Traceback (most recent call last):
  File "/tmp/ipykernel_30823/1722773412.py", line 18, in <module>
    hang_if_no_space(s)
    ~~~~~~~~~~~~~~~~^^^
  File "/tmp/ipykernel_30823/1722773412.py", line 7, in hang_if_no_space
    while True:
          ^^^^
  File "/home/nya/workspace/huge-security-books/src/fuzz/books/codes/Timeout.py", line 38, in timeout_handler
    raise TimeoutError()
TimeoutError (expected)


In [7]:
secrets = ("<space for reply>" + fuzzer(100) +
           "<secret-certificate>" + fuzzer(100) +
           "<secret-key>" + fuzzer(100) + "<other-secrets>")

uninitialized_memory_marker = "deadbeef"
while len(secrets) < 2048:
    secrets += uninitialized_memory_marker

In [9]:
def heartbeat(reply: str, length: int, memory: str) -> str:
    # Store reply in memory
    memory = reply + memory[len(reply):]

    # Send back heartbeat
    s = ""
    for i in range(length):
        s += memory[i]
    return s

In [11]:
heartbeat("potato", 6, memory=secrets)

'potato'

In [13]:
heartbeat("bird", 4, memory=secrets)

'bird'

In [15]:
heartbeat("hat", 500, memory=secrets)

'hatace for reply>6&7$+>1 88<*"<1%*56 /!(63114 >:22 >9#7987&:5)$:7#277$)>,6&(-7*:5?%.>-/==,11;2!++6<secret-certificate>="(#(22 48>8+7)":==9,:9&##(1.+89$*4%\')/5:5>&\'-8<0?<0!9\'!-)*5:1+:\'<8 5&=>:%6%-!5  !8.8<secret-key>67-1 :."=+&>!4,"520?,>;<7 =$=<other-secrets>deadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefd'

In [19]:
from ExpectError import ExpectError
with ExpectError():
    for i in range(10):
        s = heartbeat(fuzzer(), random.randint(1, 500), memory=secrets)
        assert not s.find(uninitialized_memory_marker)
        assert not s.find("secret")

Traceback (most recent call last):
  File "/tmp/ipykernel_30823/37361401.py", line 5, in <module>
    assert not s.find(uninitialized_memory_marker)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError (expected)


In [ ]:
from Fuzzer import *

cat = ProgramRunner(program="cat")
cat.run("hello")

random_fuzzer = RandomFuzzer(min_length=20, max_length=20)
for i in range(10):
    print(random_fuzzer.fuzz())

for i in range(10):
    inp = random_fuzzer.fuzz()
    result, outcome = cat.run(inp)
    assert result.stdout == inp
    assert outcome == Runner.PASS

random_fuzzer.run(cat)

random_fuzzer.runs(cat, 10)

!4&)4+").80!)!17'0/7
-:?640%4961;928:() "
62+7:,=(<"#,1$)9'69/
<.:$(&?:>6.+#4,>7628
 * '%?758! +)#(#-1>1
1*##1367#:,;!1;#8::0
9 <127'//0,33=*> 64,
+<(&!-&$;)3&+<."+636
-5>->899)('%"3$4--3*
2585$21'9%/(=049=;4;


(CompletedProcess(args='cat', returncode=0, stdout=')9*0:#"*(".9=!.=%%)1', stderr=''),
 'PASS')